In [1]:
from math import sqrt, log, pi, ceil, floor
import time

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Ubuntu Mono"
plt.rcParams['font.size'] = 9

%matplotlib auto

from earthing import *

## Genric functions

def show(var_name, var):
    print(var_name, ' = ', round(var,2))

Using matplotlib backend: TkAgg


In [33]:
# define i/p parameters
strip_width = 0.025
strip_depth = 0.005
strip_eq_radius = (strip_width + strip_depth)*2/(2*pi)  # Horizontal earth strip
radius_rod = 0.04  # Vertical earth strip for earth plate
h = 0.1
Lx = 10
Ly = 5
Lr = 1
Nx = 2 
Ny = 2
Nr = 4
Le = (Lx + Ly)*2
S = (Lx + Ly)*2 / Lr

rho = 100
xlim=(-5, Lx+5)
ylim=(-5, Ly+5)
zlim=(-5, 2)
grid=(25,25)
delta=0.2
Ig = 1000

# Define network
network = Network(rho)
# network.add_strip([0, 0, -h], [L, 0, -h], strip_width)
# network.add_mesh([0.1,0.1,-h], Lx, Ly, Nx, Ny, strip_width)
network.add_rod([0,0,-h], radius_rod, Lr)
# network.add_rod([Lx,0,-h], radius_rod, Lr)
# network.add_rod([0,Ly,-h], radius_rod, Lr)
# network.add_rod([Lx,Ly,-h], radius_rod, Lr)
# network.add_plate([-1,-1,-3], rho, 0.6, 0.6, n_cap=[0,1,0], h_cap=[0,0,1])  # Vertical plate

In [34]:
# Solve network
start_time = time.time()
network.generate_model(delta)
print("--- %s seconds ---" % (time.time() - start_time))
A1 = np.copy(network.A)

start_time = time.time()
network.generate_model_fast(delta)
print("--- %s seconds ---" % (time.time() - start_time))
A2 = network.A

# network.plot_geometry_3d(xlim=xlim, ylim=ylim, zlim=zlim, ground=True, ground_pot=False, current_distribution=False)

--- 0.005144596099853516 seconds ---
--- 0.0025391578674316406 seconds ---


In [35]:
network.solve_model()
res = network.get_resistance()
# display results
print('Value of earth resistance = ', str(res),' Ohm')

Value of earth resistance =  56.08424047030536  Ohm


In [37]:
# R = resistance_grid_with_rods(rho, A=Lx*Ly, L_E=Le, L_R=Lr, N=Nr, S=S, d=radius_rod*2, w=strip_width)
# R = resistance_grid(rho, A=Lx*Ly, L=Le, h=h) 
# Em, Es = e_mesh_step_grid(rho, Lx, Ly, Lr, Nx, Ny, Nr, strip_eq_radius*2, h, Ig)
# print(R)
# print(Em)
# print(Es)

resistance_pipe(rho, Lr, radius_rod*2)
# resistance_strip(rho, L, h, strip_width)

57.378065578753244

In [38]:
start_time = time.time()
network.solve_surface_potential(Ig, grid=grid, xlim=xlim, ylim=ylim)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
network.solve_surface_potential_fast(Ig, grid=grid, xlim=xlim, ylim=ylim)
print("--- %s seconds ---" % (time.time() - start_time))
network.plot_geometry_3d(xlim=xlim, ylim=ylim, zlim=zlim, ground=False, ground_pot=True)

--- 0.22649168968200684 seconds ---
--- 0.0011734962463378906 seconds ---


In [28]:
# network.plot_surface_potential(xlim=xlim, ylim=ylim)

In [39]:
network.gpr(Ig)

56084.24047030536

In [41]:
# Touch voltage at fence 2m away
network.get_point_potential([0,-2,0], Ig) - network.get_point_potential([0,-3,0], Ig)

2367.3769208768726

In [42]:
# Mesh voltage
network.gpr(Ig) - network.get_point_potential([Lx/Nx,Ly/Ny,0], Ig)

53258.3059661147

In [43]:
# Step voltage
loc = np.array([-Lx, -Ly, 0])
loc = loc/norm(loc)
network.get_point_potential([0,0,0], Ig) - network.get_point_potential(loc, Ig)

23146.33896766483